In [1]:
import os
import sys
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
from tasks import check

Using gpu device 0: Tesla K20Xm (CNMeM is disabled, CuDNN 3007)
/home/mcherti/work/code/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.19.
  "This module will be removed in 0.19.", DeprecationWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
v = check(what="notebook", 
          filename="../jobs/results/a5eedb043b15ceb54c0073580f47d05d/model.pkl", 
          dataset="digits")

Loading data...


INFO:tasks:Loading data...


Loading the model...


INFO:tasks:Loading the model...


(None, 1, 28, 28)
Compiling the model...


INFO:tasks:Compiling the model...


Traceback (most recent call last):
  File "/home/mcherti/work/code/feature_generation/ipynb/../tasks.py", line 682, in check
    ret = func(capsule, data, layers, w, h, c, folder, **p)
TypeError: notebook() takes exactly 6 arguments (8 given)


/home/mcherti/work/code/Lasagne/lasagne/layers/helper.py:212: UserWarning: get_output() was called with unused kwargs:
	determnistic (perhaps you meant deterministic)
  % "\n\t".join(suggestions))


In [3]:
capsule, data, layers, w, h, c = v
print(w, h, c)
print(layers.keys())
#print(w, h, c)
#print(data.X.min(), data.X.max())

TypeError: 'NoneType' object is not iterable

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
import numpy as np
for name, layer in layers.items():
    #print(name, layer.output_shape)
    #if hasattr(layer, "W"):
    #    print(layer.W.get_value().shape)
    if hasattr(layer, "W"):
        W = layer.W.get_value()
        if len(W.shape)==2:
            nbpixels = c*w*h
            if W.shape[0] == nbpixels:
                W = W.T
            a, b = w, h
        else:
            W = W.reshape((W.shape[0]*W.shape[1], W.shape[2], W.shape[3]))
            a, b = W.shape[1:]
        fig = plt.figure(figsize=(10, 10))
        s = int(np.sqrt(W.shape[0]))
        plt.axis('off')
        img=tile_raster_images(W, (a, b), (s, s))
        plt.imshow(img, cmap="gray", interpolation='none')
        plt.show()

In [ ]:
from skimage.filters import threshold_otsu
np.random.seed(12345)
T = [0.5, 0.7, 0.8, 0.85, 0.87, 0.89, 0.9, 0.91, 0.92]
def binarize(X):
    for i in range(X.shape[0]):
        X[i] = X[i] >= threshold_otsu(X[i])
    return X
s = np.random.uniform(size=(100, c, w,h ))
s = d
#s = s > 0.2
#s = s > 0.7
s = s.astype(np.float32)
loss = []
for i in range(200):
    sprev = s
    #s *= np.random.uniform(size=s.shape)<=0.9999
    s = capsule.reconstruct(s)
    s = s  > (1-0.2)
    s = s.astype(np.float32)
    score = np.abs(s - sprev).sum()
    print(score)
    loss.append(score)
    if score == 0:
        break
    loss.append(score)


In [ ]:
from lasagnekit.misc.plot_weights import dispims_color

s = s.astype(np.float32)
plt.plot(loss)
print("Nb of iterations", len(loss))
sz = int(np.sqrt(s.shape[0]))
fig = plt.figure(figsize=(10, 10))


if s.shape[1] == 1:
    print(s.dtype)
    img = tile_raster_images(s[:, 0], (w, h), (sz, sz), tile_spacing=(2, 2), )
    plt.imshow(img, cmap="gray", interpolation='none')
    plt.axis('off')
    plt.show()
else:
    ss = s.transpose((0, 2, 3, 1))
    print(s.shape)
    img = dispims_color(ss)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

"""
import hashlib
from collections import Counter
def hash_binary_vector(x):
    m = hashlib.md5()
    ss = str(x.flatten().tolist())
    m.update(ss)
    return m.hexdigest()
def hash_matrix(X):
    hashes = []
    for i in range(X.shape[0]):
        h = hash_binary_vector(X[i])
        hashes.append(h)
    return hashes
cnt = Counter(hash_matrix(s))
print(len(cnt))
print(s.shape)
V = sorted(cnt.values(), reverse=True)
plt.bar(np.arange(len(V)), V)
plt.ylabel("frequency")
plt.xlabel("fixed point")
plt.legend()
plt.xlim((0, len(cnt)))
#plt.title("Frequency of fixed points")
"""